In [1]:
import tensornetwork as tn
import numpy as np

In [2]:
def block1D(*dimensions, normal = True):
    """Construct a new matrix for the MPS with random numbers from 0 to 1"""
    size = tuple([x for x in dimensions])
    if normal:
        A = np.random.normal(size=size)
    else:
        A = np.random.random_sample(size)
    A = (A.transpose(2, 1, 0) + A) / 2
    return A




def create_MPS(n, A):
    """Build the MPS tensor"""
    mps = [tn.Node(np.copy(A)) for _ in range(n)]
    # connect edges to build mps
    connected_edges = []
    for k in range(0, n):
        conn = mps[k][2] ^ mps[(k + 1) % n][0]
        connected_edges.append(conn)
    return mps, connected_edges


# Check injectivity with random matrix

In [6]:
bond_dim = 2
sps = 4
A = block1D(bond_dim, sps, bond_dim)

In [7]:
A_ = A.transpose(1,0,2)
B = A_.reshape(sps,bond_dim**2)

In [8]:
U, S, V = np.linalg.svd(B)
print("Injective ? ",  np.sum(np.abs(S) > 1E-10) == bond_dim**2)

Injective ?  False


### SIngle matrix cannot be injective for this sps and bd case
But when we use blocked matrix with l = 2, it achieve injectivity. (Injective length is 2)

In [9]:
A2 = np.einsum("ijk,klm->jlim",A,A)
A2_ = A2.reshape(sps**2, bond_dim**2)
U, S, V = np.linalg.svd(A2_)
print("Injective ? ",  np.sum(np.abs(S) > 1E-10) == bond_dim**2)

Injective ?  True


### Check kernel of reduced density matrix is same as kernel of bocked matrix with l = 2

calculate MPS

In [10]:
mps_nodes, mps_edges = create_MPS(6, A)
for k in range(len(mps_edges)):
    C = tn.contract(mps_edges[k])
y = C.tensor.reshape(sps**2, -1)
rho = y @ y.T
E, V = np.linalg.eigh(rho)
Vp = V[:, (np.abs(E) < 1E-10)]
P = Vp @ Vp.T

calculate with A2

In [11]:
U, s, V = np.linalg.svd(A2_)
Up = U[:, len(s):]
Up2 = U[:, :len(s)]
h = Up @ Up.T
h2 = Up2 @ Up2.T

In [12]:
print("two projector are the same? : ",np.linalg.norm(h - P))

two projector are the same? :  5.315654982393576e-15


This means we can use both kernel to construct ff model.

In [13]:
ya = np.abs(y)
rho = y @ y.T
rho_a = ya @ ya.T
print("absolute of reduced matrix is same as reduce matrix of absolute state? : ", np.all(np.abs(rho) == rho_a))

absolute of reduced matrix is same as reduce matrix of absolute state? :  False


## AKLT

In [15]:
A = np.zeros((3, 2, 2))
A[0][1,0] = -1/np.sqrt(2)
A[2][0,1] = 1/np.sqrt(2)
A[1][0,0] = 1/2
A[1][1,1] = -1/2
A_ = A.transpose(1, 0, 2)

In [16]:
AKLT_B = A.reshape(3, 4)
U, S, V = np.linalg.svd(AKLT_B)
print("Injective ? ",  np.sum(np.abs(S) > 1E-10) == 2**2)

Injective ?  False


In [17]:
mps_nodes, mps_edges = create_MPS(6, A_)
for k in range(len(mps_edges)):
    C = tn.contract(mps_edges[k])
y = C.tensor.reshape((3,) * 6)
ya = np.abs(y)
rho = y @ y.T
rho_a = ya @ ya.T

In [18]:
print("absolute of reduced matrix is same as reduce matrix of absolute state? : ", np.all(np.abs(rho) == rho_a))

absolute of reduced matrix is same as reduce matrix of absolute state? :  True


## Test concept for seed = 0

In [85]:
import numpy as np
import random
import utils
np.random.seed(0)
random.seed(0)
path = "array/torch/FF1D_loc/s_3_r_2_lt_1_d_1_seed_0/original_mel_LION/lr_0.005_epoch_20000/"
h = np.load(path + "H/0.npy")
u = np.load(path + "loss_0.17387/u/0.npy")
A = block1D(2,3,2)

In [86]:
sps = 3
mps_nodes, mps_edges = create_MPS(6, A)
for k in range(len(mps_edges)):
    C = tn.contract(mps_edges[k])
y = C.tensor.reshape(sps**2, -1)
y /= np.linalg.norm(y)
x = y.reshape(-1)

In [87]:
L = 6
U = np.kron(u,u)
H = -utils.sum_ham(h, [[i, (i+1)%L] for i in range(L)], L, 3)
HU = -utils.sum_ham(U @ h @ U.T, [[i, (i+1)%L] for i in range(L)], L, 3)

In [88]:
print("check if MPS is gs of H",x @ H @ x)

check if MPS is gs of H 5.575987107248906e-18


In [89]:
ya = np.abs(y)
rho_a = ya @ ya.T
rho = y @ y.T
print("absolute of reduced matrix is same as reduce matrix of absolute state? : ", np.all(np.abs(rho) == rho_a))

absolute of reduced matrix is same as reduce matrix of absolute state? :  False


In [90]:
EU, VU = np.linalg.eigh(HU)
EUA, VUA = np.linalg.eigh(utils.stoquastic(HU))
eua, vua = np.linalg.eigh(utils.stoquastic(-U @ h @ U.T))
print(EUA[:4])
print(EU[:4])
print("system energy < local energy * L :",np.abs(EUA[0]) < np.abs(eua[0] * L))

[-0.83925147 -0.13908918 -0.09866087 -0.09866087]
[6.60797479e-16 1.44099473e-02 4.75701599e-02 9.52892885e-02]
system energy < local energy * L : True


In [53]:
print(np.linalg.eigvalsh(-h)[:10])
print(np.linalg.eigvalsh(utils.stoquastic(-U @ h @ U.T))[:10])

[-1.21061520e-16 -5.85601514e-17  2.40630362e-17  1.55558635e-16
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00]
[-0.1738801   0.02115874  0.03146863  0.15053682  0.95557734  0.97187827
  0.99080875  1.02395328  1.02849826]


In [60]:
print("similarity between abs gs and original gs",np.abs(VU[:,0]) @ VUA[:,0])

similarity between abs gs and original gs 0.8214617847419597


## Test concept for seed = 1

In [105]:
np.random.seed(1)
random.seed(1)
path = "array/torch/FF1D_loc/s_3_r_2_lt_1_d_1_seed_1/original_mel_LION/lr_0.005_epoch_20000/"
h = np.load(path + "H/0.npy")
u = np.load(path + "loss_0.00000/u/0.npy")
A = block1D(2,3,2)

In [106]:
sps = 3
mps_nodes, mps_edges = create_MPS(6, A)
for k in range(len(mps_edges)):
    C = tn.contract(mps_edges[k])
y = C.tensor.reshape(sps**2, -1)
y /= np.linalg.norm(y)
x = y.reshape(-1)

In [107]:
L = 6
U = np.kron(u,u)
H = -utils.sum_ham(h, [[i, (i+1)%L] for i in range(L)], L, 3)
HU = -utils.sum_ham(U @ h @ U.T, [[i, (i+1)%L] for i in range(L)], L, 3)

In [108]:
print("check if MPS is gs of H",x @ H @ x)

check if MPS is gs of H 1.1414333081516405e-16


In [109]:
ya = np.abs(y)
rho_a = ya @ ya.T
rho = y @ y.T
print("absolute of reduced matrix is same as reduce matrix of absolute state? : ", np.all(np.abs(rho) == rho_a))

absolute of reduced matrix is same as reduce matrix of absolute state? :  False


In [110]:
EU, VU = np.linalg.eigh(HU)
EUA, VUA = np.linalg.eigh(utils.stoquastic(HU))
eua, vua = np.linalg.eigh(utils.stoquastic(-U @ h @ U.T))
print(EUA[:4])
print(EU[:4])
print("system energy < local energy * L :",np.abs(EUA[0]) < np.abs(eua[0] * L))

[-7.22298599e-06 -2.03072481e-06 -1.73025351e-06 -8.12094903e-07]
[-3.66817746e-15  3.80808374e-08  4.31880625e-08  6.29862947e-08]
system energy < local energy * L : True


In [111]:
print(np.linalg.eigvalsh(-h)[:10])
print(np.linalg.eigvalsh(utils.stoquastic(-U @ h @ U.T))[:10])
print(np.linalg.eigvalsh(-U @ h @ U.T)[:10])

[-1.28693374e-16 -3.61496388e-17  1.24668866e-17  2.13632818e-16
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00]
[-1.41069773e-06  2.59445219e-09  2.01728866e-07  1.20637616e-06
  9.99999957e-01  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000004e+00]
[-2.15712124e-17 -1.69051265e-17  4.04807782e-17  1.13307289e-16
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00]


In [112]:
print("similarity between abs gs and original gs",np.abs(VU[:,0]) @ VUA[:,0])

similarity between abs gs and original gs -0.7489758709633559


In [113]:
A

array([[[ 1.62434536,  0.56652768],
        [-0.52817175, -0.37696476],
        [ 0.86540763, -0.41971538]],

       [[ 0.56652768, -0.7612069 ],
        [-0.37696476, -0.24937038],
        [-0.41971538, -2.06014071]]])

## Test concept for seed = 2

In [92]:
np.random.seed(2)
random.seed(2)
path = "array/torch/FF1D_loc/s_3_r_2_lt_1_d_1_seed_2/original_mel_LION/lr_0.005_epoch_20000/"
h = np.load(path + "H/0.npy")
u = np.load(path + "loss_0.00003/u/0.npy")
A = block1D(2,3,2)

In [93]:
sps = 3
mps_nodes, mps_edges = create_MPS(6, A)
for k in range(len(mps_edges)):
    C = tn.contract(mps_edges[k])
y = C.tensor.reshape(sps**2, -1)
y /= np.linalg.norm(y)
x = y.reshape(-1)

In [94]:
L = 6
U = np.kron(u,u)
H = -utils.sum_ham(h, [[i, (i+1)%L] for i in range(L)], L, 3)
HU = -utils.sum_ham(U @ h @ U.T, [[i, (i+1)%L] for i in range(L)], L, 3)

In [95]:
print("check if MPS is gs of H",x @ H @ x)

check if MPS is gs of H -1.7024966217198452e-17


In [96]:
ya = np.abs(y)
rho_a = ya @ ya.T
rho = y @ y.T
print("absolute of reduced matrix is same as reduce matrix of absolute state? : ", np.all(np.abs(rho) == rho_a))

absolute of reduced matrix is same as reduce matrix of absolute state? :  False


In [101]:
EU, VU = np.linalg.eigh(HU)
EUA, VUA = np.linalg.eigh(utils.stoquastic(HU))
eua, vua = np.linalg.eigh(utils.stoquastic(-U @ h @ U.T))
print(EUA[:4])
print(EU[:4])
print("system energy < local energy * L :",np.abs(EUA[0]) < np.abs(eua[0] * L))

[-1.25554735e-04 -3.53733998e-06  1.20532536e-05  1.87710922e-05]
[-3.97531403e-16  3.08889596e-08  1.91522648e-06  6.87843956e-06]
system energy < local energy * L : True


In [102]:
print(np.linalg.eigvalsh(-h)[:10])
print(np.linalg.eigvalsh(utils.stoquastic(-U @ h @ U.T))[:10])
print(np.linalg.eigvalsh(-U @ h @ U.T)[:10])

[-1.58570710e-16 -5.24109144e-18  1.57484442e-16  2.28371965e-16
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00]
[-2.53382271e-05  1.02204863e-06  4.29662871e-06  2.00200875e-05
  9.99998110e-01  9.99998268e-01  1.00000000e+00  1.00000172e+00
  1.00000190e+00]
[-2.27460895e-16 -4.32496448e-17  3.01862446e-18  4.56473109e-17
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00]


In [103]:
print("similarity between abs gs and original gs",np.abs(VU[:,0]) @ VUA[:,0])

similarity between abs gs and original gs -0.7342053037197074


In [104]:
A

array([[[-0.41675785,  0.22330729],
        [-2.1361961 ,  0.29115929],
        [-1.79343559, -0.14514666]],

       [[ 0.22330729, -1.24528809],
        [ 0.29115929, -0.90900761],
        [-0.14514666,  2.29220801]]])

## seed 3

In [115]:
np.random.seed(3)
random.seed(3)
path = "array/torch/FF1D_loc/s_3_r_2_lt_1_d_1_seed_3/original_mel_LION/lr_0.005_epoch_20000/"
h = np.load(path + "H/0.npy")
u = np.load(path + "loss_0.26422/u/0.npy")

In [116]:
L = 6
U = np.kron(u,u)
H = -utils.sum_ham(h, [[i, (i+1)%L] for i in range(L)], L, 3)
HU = -utils.sum_ham(U @ h @ U.T, [[i, (i+1)%L] for i in range(L)], L, 3)

In [118]:
EU, VU = np.linalg.eigh(HU)
EUA, VUA = np.linalg.eigh(utils.stoquastic(HU))
eua, vua = np.linalg.eigh(utils.stoquastic(-U @ h @ U.T))
print(EUA[:4])
print(EU[:4])
print(np.linalg.eigvalsh(-h)[:4])
print(np.linalg.eigvalsh(utils.stoquastic(-U @ h @ U.T))[:4])
print("system energy < local energy * L :",np.abs(EUA[0]) < np.abs(eua[0] * L))

[-1.24706448 -0.38915866 -0.21792774 -0.21792774]
[3.79624928e-15 2.24246573e-02 1.08197562e-01 1.89560221e-01]
[-4.11276158e-17  2.67788601e-17  8.29393041e-17  1.35228564e-16]
[-0.26423722  0.02156496  0.06297436  0.25055308]
system energy < local energy * L : True
